# Orientações

A primeira parte do trabalho possui o propósito de familiarizar o aluno a algumas ferramentas como Python, Pandas e Gurobi

Ao ser requisitado um modelo matemático para ilustrar uma situação proposta em uma questão, defina claramente os conjuntos, parâmetros, variáveis de decisão, função objetivo e restrições. Ao ser requisitado que um modelo seja alterado, não é necessário reescrevê-lo por completo, somente aponte as restrições que devem ser alteradas/removidas/acrescentadas.

Explique os motivos para a elaboração de suas restrições/FO. Mostre quando elas são ativadas e levante hipóteses sobre como elas podem interferir nas respostas que o modelo fornecerá.

Ao ser pedido que uma solução seja explicada, espera-se que sejam levantadas hipóteses sobre a resposta obtida, relacionando-se com o modelo matemático e os dados utilizados. Além da própria solução, também imprima dados relevantes como parâmetros e outras variáveis de decisão que permitam compreender mais profundamente a solução proposta pelo resolvedor.

Sempre analise (discorra) as suas respostas!

Avisos:

O Gurobi lida com os números a partir da ideia de precisão. Por exemplo, uma variável que vale "1" pode ser expressa por 1.00000 ou por 0.99999 (pois está dentro de uma precisão/tolerância de 0.00001). Devido a precisão, algumas de suas respostas "-0.0", equivalente a zero. Enfim, tome cuidado ao comparar a resposta obtida pelo resolvedor com uma expressão, por exemplo "x == 1", nem sempre será verdadeiro, como no caso de "x = 0.99999"

Você pode mudar parâmetros do método "setParam" do modelo do Gurobi, por exemplo: m.setParam("parâmetro", "valor")


# Exemplo

In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

Inês planeja montar um arranjo de flores para o aniversário de seu pai. Ao ir para a floricultura Fictícia, ela recebeu um menu com diversas espécies e algumas informações relevantes sobre elas. O buquê será composto pela floricultura de acordo com as espécies que Inês escolher

O menu fornecido pela floricultura encontra-se no arquivo "flores_a.csv"

O arquivo contém as seguintes colunas:



*   Espécie: indica uma espécie de flor
*   Raridade: a escassez da espécie em floriculturas. É uma escala de zero a dez com precisão de duas casas decimais
*   Finalidade: o principal fim que aquela espécie de flor possui
*   Intensidade_Perfume: o quão perfumada é a espécie de flor. É uma escala de zeroa a dez com precisão de uma casa decimal
*   Pétalas: quantidade média de pétalas de flor de uma espécie
*   Preço: valor da unidade da flor em reais

O arquivo já está pronto para uso, não sendo necessário eliminar linhas ou colunas por problemas na coleta e compilação dos dados




Vamos ler o arquivo de dados

In [2]:
#vamos definir uma função que sempre leia o arquivo e retorne o dataset, assim, em qualquer célula podemos invocar o dataset sem qualquer alteração

def ret_dataset():
  return pd.read_csv("flores_a.csv")

df = ret_dataset()
print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'flores_a.csv'

## Item Único

Auxilie Inês a escolher as melhores espécies de flores para o arranjo elaborando um modelo de otimização que maximize a intensidade de perfume das plantas. Ela escolherá oito espécies de flores para compor  o buquê

### a) Defina um modelo para ajudar na tomada de decisão de Inês

Conjuntos\
$F →$ Conjunto de espécies de flores

Parâmetros:\
$I_f →$ Intensidade de perfume de uma flor $f \in F$

Variáveis de decisão:\
$x_f →$ Binária. Indica se a espécie de flor $f \in F$ deve ser incluída no buquê

Função Objetivo:\
max $\sum_{f \in F} I_fx_f$

Restrições:\
$\sum_{f \in F} x_f = 8$

### b) Resolva o modelo com um resolvedor

In [ ]:
#obtemos o dataset
df = ret_dataset()

#criamos o modelo
m = gp.Model()

#definimos os conjuntos
F = list(df["especie"])

#definimos os parâmetros
I_f = list(df["intensidade_perfume"])

#definimos a variável de decisão x
x = []
for i in range(len(F)):
  x.append(m.addVar(vtype=GRB.BINARY, name=F[i]))

#definimos a função objetivo
m.setObjective(gp.quicksum(x[i]*I_f[i] for i in range(len(F))), GRB.MAXIMIZE)

#definimos as restrições
m.addConstr(gp.quicksum(x[i] for i in range(len(F))) == 8)

#resolvemos o modelo
m.optimize()

#apontamos quais espécies de flores devem ser escolhidas
print("Espécie do buquê | Intensidade de perfume")
especies_no_buque = 0
for i in range(len(F)):
  if x[i].x == 1:
    print(F[i], I_f[i])
    especies_no_buque += 1
print("Espécies de flores disponíveis: ", len(F))
print("Espécies de flores escolhidas: ", especies_no_buque)
print("Valor FO:", m.objVal)
print("É a solução ótima?", m.Status == GRB.OPTIMAL)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1 rows, 157 columns and 157 nonzeros
Model fingerprint: 0x537ede44
Variable types: 0 continuous, 157 integer (157 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+00, 8e+00]
Found heuristic solution: objective 45.1000000
Presolve removed 1 rows and 157 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 2: 79.1 45.1 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.910000000000e+01, best bound 7.910000000000e+01, gap 0.0000%
Espécie do buquê | Intensidade de perfume
Alecrim 9.8
Cravo 10